In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns

In [2]:
# from google.colab import drive
# drive.mount('/content/drive')
# %cd /content/drive/MyDrive/Cancer_Hints_Project/

In [3]:
random_state = 1234

In [5]:
all_hints = pd.read_csv("../Data/hints_filtered0.csv")

In [6]:
print(all_hints.shape)
all_hints.head()

(3630, 300)


,Stratum,HIGHSPANLI,HISPSURNAME,HISP_HH,Treatment_C2,RUC2003,CENSDIV,CENSREG,FormType,Language_Flag,...,AgeGrpA,AgeGrpB,EducA,EducB,RaceEthn,HHInc,BMI,smokeStat,PHQ4,IncomeRanges_IMP
0,HM,2,2,2,1,1,5,3,2,1,...,1.0,1.0,3.0,3.0,2.0,5.0,29.0,1.0,2.0,7.0
1,LM,2,2,2,1,6,2,1,2,1,...,4.0,4.0,2.0,2.0,2.0,NaN,27.0,2.0,1.0,4.0
2,HM,2,2,2,1,1,3,2,2,1,...,1.0,1.0,4.0,4.0,5.0,5.0,20.0,1.0,2.0,8.0
3,HM,2,2,2,1,1,5,3,2,1,...,3.0,2.0,4.0,5.0,3.0,5.0,22.0,3.0,1.0,9.0
4,HM,2,2,2,1,4,5,3,2,1,...,4.0,2.0,NaN,NaN,2.0,3.0,28.0,3.0,0.0,5.0


In [7]:
# Assuming df is your DataFrame
non_numerical_columns = all_hints.select_dtypes(exclude=['number']).columns

print("Columns with non-numerical inputs:")
print(non_numerical_columns)
all_hints = all_hints.drop(columns=non_numerical_columns, axis=1, inplace=False)


Columns with non-numerical inputs:
Index(['Stratum', 'WhereSeekHealthInfo_OS', 'StrongNeedCancerInfo_OS',
       'Internet_OtherOS', 'InternetCancerInfoSite_OS', 'CaOther_OS',
       'OccupationStatus_OS', 'VAR_STRATUM', 'UpdateDate'],
      dtype='object')


In [8]:
# print(all_hints.isna().sum())
na_counts = all_hints.isna().sum()
features_to_remove = na_counts[na_counts > 500] #remove var with more than 500 na
feature_names_to_remove = features_to_remove.index.tolist()  # Get the names of these features
print(len(feature_names_to_remove))

53


In [9]:
hints = all_hints.drop(columns=feature_names_to_remove, axis=1, inplace=False)
hints = hints.dropna()
print(hints.shape)

(1413, 238)


## Fitting a Random Forest

In [10]:
from sklearn.model_selection import train_test_split
from sklearn.feature_selection import RFE, RFECV

from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestClassifier
from sklearn.pipeline import make_pipeline

from sklearn.model_selection import StratifiedKFold, cross_val_score


In [11]:
X,y = hints.drop(["EverHadCancer"], axis=1), hints.EverHadCancer

In [12]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=random_state, stratify=y)


In [13]:
rf =  RandomForestClassifier(random_state=random_state)
kfold = StratifiedKFold(n_splits=5, shuffle=True, random_state=random_state)


In [14]:

accuracies = []
# Perform k-fold cross-validation
for train_index, cv_index in kfold.split(X_train, y_train):
    trainX, cvX = X_train.iloc[train_index], X_train.iloc[cv_index]
    trainY, cvY = y_train.iloc[train_index], y_train.iloc[cv_index]

    # Fit the pipeline on the training data
    rf.fit(trainX, trainY)

    # Evaluate the model on the cross-validation data
    accuracy = rf.score(cvX, cvY)
    accuracies.append(accuracy)


# Calculate the average accuracy
av_accuracies = np.mean(accuracies)
print(f'Average cross-validation accuracy: {av_accuracies}')




Average cross-validation accuracy: 0.8991150442477875
